
# PBH-Aware Identifiability Demo (Single-Trajectory, Zero-Input)

This notebook reproduces the constructions from our discussion and generates the figures & diagnostics:
- A dense symmetric pair \( (A,B) \) with \(B \parallel v_1\) and an **uncontrollable** direction.
- A perturbed pair \( (\tilde A,\tilde B) \) with the **same** eigenvectors and \( \tilde B=B \), but a different second eigenvalue.
- Two initial states: a **bad** one \(x_0^{\text{bad}}=v_3\) that hides the ambiguity, and a **good** one \(x_0^{\text{good}}=v_2+v_3\) that exposes it.

We compare:
1) the **structured distance to PBH failure**,
2) **Krylov generator** proxies,
3) **time-limited controllability Gramian** proxies, and
4) the **left-eigenvector criterion**,

and we visualize eigenspaces, projections, and trajectories in \([x_1,x_2]\) and \([x_2,x_3]\) planes.



## Theory (compact)

Let \( \mathcal H(\lambda) := [\,\lambda I - A\; B\,] \in \mathbb{C}^{n\times(n+m)} \), and define the augmented pencil
\(\mathcal H_{\text{aug}}(\lambda) := [\,\lambda I - A\; B\; x_0\,]\).  
PBH failure of \(\mathcal H_{\text{aug}}\) occurs iff there exists \(0\neq w\) such that
\(w^\top x_0 = 0\) and \(w^\top [\,\lambda I - A\; B\,] = 0\).

Let \(P_{x_0^\perp} := I - \frac{x_0 x_0^\top}{\|x_0\|_2^2}\) be the orthogonal projector onto \(x_0^\perp\), and let \(Q\) be any
orthonormal basis of \(x_0^\perp\) (so \(QQ^\top = P_{x_0^\perp}\)).  
The **structured Frobenius distance** that leaves \(x_0\) untouched is
\[
d_\mathrm{Frob}(A,B\,\|\,x_0)
~=~\inf_{\lambda\in\mathbb{C}}~\sigma_{\min}\!\big(P_{x_0^\perp}\,[\,\lambda I-A\; B\,]\big)
~=~\inf_{\lambda\in\mathbb{C}}~\sigma_{\min}\!\big(Q^\top \mathcal H(\lambda)\big).
\]
*Sketch.* Since PBH failure for the augmented pencil is equivalent to existence of a nonzero
\(w \in x_0^\perp\) with \(w^\top \mathcal H(\lambda)=0\), restricting to \(x_0^\perp\) by left-multiplication with \(Q^\top\) isolates the relevant rows. The smallest Frobenius-norm perturbation (restricted to those rows) that drops rank equals
\(\sigma_{\min}(Q^\top \mathcal H(\lambda))\) by Eckart–Young; taking \(\inf_\lambda\) yields the claim.  
If \(x_0=0\Rightarrow P_{x_0^\perp}=I\), this recovers the classical distance to uncontrollability.

**Other proxies.**
- **Krylov generator** \(K_n(A,[x_0,B]) = [\,x_0,B,Ax_0,AB,\dots,A^{n-1}x_0,A^{n-1}B\,]\)  
  use \(\sigma_{\min}(K_n),~\kappa_2(K_n),~\mathrm{rank}(K_n)\).
- **Time-limited Gramian** \(W_T = \int_0^T e^{At}[x_0~B][x_0~B]^\top e^{A^\top t}\,dt\)  
  use \(\lambda_{\min}(W_T),~\kappa_2(W_T)\).
- **Left-eigenvector criterion.**
  For \(w_i^\top A = \lambda_i w_i^\top\), define
  \(\mu_i = \frac{\|w_i^\top [x_0~B]\|_2}{\|w_i\|_2}\) and \(\mu_{\min}=\min_i \mu_i\).

**One-parameter family.** With \(x_0(\theta)=\cos\theta\,v_2+\sin\theta\,v_3\), we have
\(\mu_2=|\cos\theta|\), \(\mu_3=|\sin\theta|\), hence \(\mu_{\min}=\min\{|\cos\theta|,|\sin\theta|\}\).
Empirically \(d_\mathrm{Frob}(A,B\,\|\,x_0(\theta))\) co-moves with \(\mu_{\min}\) and vanishes at the endpoints.



## Concrete pairs and initial states

We use an orthonormal eigenbasis \(Q=[v_1~v_2~v_3]\):
\[
v_1=\tfrac{1}{\sqrt3}\!\begin{bmatrix}1\\1\\1\end{bmatrix},\quad
v_2=\tfrac{1}{\sqrt2}\!\begin{bmatrix}1\\-1\\0\end{bmatrix},\quad
v_3=\tfrac{1}{\sqrt6}\!\begin{bmatrix}1\\1\\-2\end{bmatrix}.
\]
Original pair:
\[
A=Q\,\mathrm{diag}(1,2,3)\,Q^\top,\qquad B=\begin{bmatrix}1\\1\\1\end{bmatrix}=\sqrt3\,v_1.
\]
Perturbed pair (same eigenvectors, different middle eigenvalue):
\[
\tilde A=Q\,\mathrm{diag}(1,5,3)\,Q^\top,\qquad \tilde B=B.
\]
Initial states:
\[
x_0^{\text{bad}}=v_3,\qquad x_0^{\text{good}}=v_2+v_3.
\]
**Closed-form zero-input solutions.**
\[
\begin{aligned}
&x_A(t)=e^{At}x_0, \qquad x_{\tilde A}(t)=e^{\tilde A t}x_0.\\[2pt]
&x_0^{\text{bad}}=v_3:\quad x_A(t)=x_{\tilde A}(t)=e^{3t}v_3\quad(\text{identical for all }t).\\[2pt]
&x_0^{\text{good}}=v_2+v_3:\quad x_A(t)=e^{2t}v_2+e^{3t}v_3,\;\; x_{\tilde A}(t)=e^{5t}v_2+e^{3t}v_3\;(\text{different for }t>0).
\end{aligned}
\]


In [ ]:

# Imports and helpers
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import eig, svd, norm
import pandas as pd

# plotting defaults (no specific colors; one plot per figure)
plt.rcParams['figure.dpi'] = 140

def proj(arr, plane="x1x2"):
    """Project vector(s) to a coordinate plane."""
    arr = np.asarray(arr)
    if arr.ndim == 1:
        if plane == "x1x2": return arr[0], arr[1]
        if plane == "x2x3": return arr[1], arr[2]
    else:
        if plane == "x1x2": return arr[:,0], arr[:,1]
        if plane == "x2x3": return arr[:,1], arr[:,2]
    raise ValueError("plane must be 'x1x2' or 'x2x3'")

def plot_eigenspaces(A, title_suffix, vectors=None, plane="x1x2"):
    w, V = eig(A)
    order = np.argsort(w.real)
    V = V[:, order].real
    w = w[order].real

    fig = plt.figure()
    ax = plt.gca()
    ax.set_aspect('equal', adjustable='box')
    ax.set_xlabel(plane[:2]); ax.set_ylabel(plane[-2:])
    ax.set_title(f"Eigenspaces ({title_suffix}) in {plane}")

    rng = np.linspace(-1.2, 1.2, 2)
    for i in range(3):
        vi = V[:, i]
        pts = np.outer(rng, vi)
        x, y = proj(pts, plane)
        ax.plot(x, y, linestyle='--', label=f"eig {i+1}: λ≈{w[i]:.2f}")

    if vectors:
        for name, vec in vectors.items():
            x, y = proj(vec, plane)
            ax.arrow(0, 0, x, y, head_width=0.05, length_includes_head=True)
            ax.text(x, y, f"  {name}")

    ax.legend(loc="best")
    ax.axhline(0, linewidth=0.5); ax.axvline(0, linewidth=0.5)
    plt.show()

def trajectories(A, x0, T=1.0, N=400):
    """Compute x(t)=e^{At}x0 via eigendecomp."""
    w, V = eig(A)
    Vinv = np.linalg.inv(V)
    t = np.linspace(0, T, N)
    X = np.zeros((N, 3))
    coeff = Vinv @ x0
    for k, tk in enumerate(t):
        eLam = np.exp(w * tk)
        X[k] = (V @ (eLam * coeff)).real
    return t, X

def plot_trajectory_pair(A, A_t, x0, title, plane="x1x2", T=1.0, N=400):
    t, X  = trajectories(A,   x0, T=T, N=N)
    _, Xt = trajectories(A_t, x0, T=T, N=N)
    fig = plt.figure()
    ax = plt.gca()
    ax.set_aspect('equal', adjustable='box')
    ax.set_xlabel(plane[:2]); ax.set_ylabel(plane[-2:])
    ax.set_title(f"{title} in {plane}")

    x, y   = proj(X,  plane)
    x2, y2 = proj(Xt, plane)
    ax.plot(x, y,  label="A",  linewidth=1.5)
    ax.plot(x2, y2, label="Ã", linewidth=1.5, linestyle=':')

    idx = int(0.8 * (len(x)-1))
    ax.annotate("", xy=(x[idx+1], y[idx+1]),   xytext=(x[idx], y[idx]),
                arrowprops=dict(arrowstyle="->"))
    ax.annotate("", xy=(x2[idx+1], y2[idx+1]), xytext=(x2[idx], y2[idx]),
                arrowprops=dict(arrowstyle="->"))

    ax.legend(loc="best")
    ax.axhline(0, linewidth=0.5); ax.axvline(0, linewidth=0.5)
    plt.show()

def K_generator(A, x0, B, n=3):
    cols = [x0, B]
    Akx, AkB = x0.copy(), B.copy()
    for _ in range(1, n):
        Akx = A @ Akx; AkB = A @ AkB
        cols.extend([Akx, AkB])
    return np.column_stack(cols)

def mu_min(A, B, x0):
    w, V = eig(A.T)
    V = V.real
    mus = []
    for i in range(3):
        wi = V[:, i]
        mus.append(norm(np.array([wi @ x0, wi @ B])))
    return float(min(mus))

def gramian_time_limited(A, G, T=1.0, N=2000):
    """W_T = int_0^T e^{At} G G^T e^{A^T t} dt (trapezoid)."""
    dt = T / N
    w, V = eig(A)
    Vinv = np.linalg.inv(V)
    GGt = G @ G.T
    W = np.zeros((3,3), dtype=float)
    for k in range(N+1):
        t = k * dt
        eLam = np.exp(w * t)
        E = (V @ np.diag(eLam) @ Vinv).real
        weight = dt * (0.5 if (k==0 or k==N) else 1.0)
        W += E @ GGt @ E.T * weight
    return W

def d_frob_structured(A, B, x0, lam_grid=None):
    """Compute min_λ σ_min(P_{x0^⊥}[λI-A  B]) over a real λ grid."""
    x0n = x0 / norm(x0)
    P = np.eye(3) - np.outer(x0n, x0n)
    if lam_grid is None:
        w = np.linalg.eigvals(A).real
        lam_grid = np.linspace(min(w)-1.0, max(w)+1.0, 2001)
    mins = []
    for lam in lam_grid:
        H = np.hstack([lam*np.eye(3) - A, B.reshape(-1,1)])
        M = P @ H
        svals = svd(M, compute_uv=False)
        mins.append(svals.min())
    idx = int(np.argmin(mins))
    return float(mins[idx]), float(lam_grid[idx])


In [ ]:

# Define eigenbasis, pairs, and initial states
from numpy.linalg import norm

v1 = np.array([1.0, 1.0, 1.0]); v1 /= norm(v1)
v2 = np.array([ 1.0,-1.0, 0.0]); v2 /= norm(v2)
v3 = np.array([ 1.0, 1.0,-2.0]); v3 /= norm(v3)
Q  = np.column_stack([v1, v2, v3])

A   = Q @ np.diag([1.0, 2.0, 3.0]) @ Q.T
A_t = Q @ np.diag([1.0, 5.0, 3.0]) @ Q.T

B = np.array([1.0, 1.0, 1.0])  # = √3 v1
x0_bad  = v3.copy()
x0_good = v2 + v3

A, A_t, B, x0_bad, x0_good


In [ ]:

# Diagnostics for good/bad x0 (for A,B)
rows = []
for name, x0 in [("x0_good", x0_good), ("x0_bad", x0_bad)]:
    K  = K_generator(A, x0, B)
    sminK = svd(K, compute_uv=False).min()
    mu    = mu_min(A, B, x0)
    W     = gramian_time_limited(A, np.column_stack([x0, B]), T=1.0, N=800)
    dF, lam_star = d_frob_structured(A, B, x0)
    rows.append({
        "case": name,
        "sigma_min(K)": float(sminK),
        "mu_min": float(mu),
        "lambda_min(W1)": float(np.linalg.eigvalsh(W).min()),
        "d_Frob": float(dF),
        "lambda* (argmin)": float(lam_star),
    })
df = pd.DataFrame(rows)
df


In [ ]:

# Eigenspace visualizations (top row)
vecs = {"B": B, "x0_good": x0_good, "x0_bad": x0_bad}
plot_eigenspaces(A,   "A",   vecs, plane="x1x2")
plot_eigenspaces(A,   "A",   vecs, plane="x2x3")
plot_eigenspaces(A_t, "Ã", vecs, plane="x1x2")
plot_eigenspaces(A_t, "Ã", vecs, plane="x2x3")


In [ ]:

# Trajectories (third row), short horizon for readability
plot_trajectory_pair(A, A_t, x0_good, "Trajectories from x0_good", plane="x1x2", T=1.0)
plot_trajectory_pair(A, A_t, x0_good, "Trajectories from x0_good", plane="x2x3", T=1.0)
plot_trajectory_pair(A, A_t, x0_bad,  "Trajectories from x0_bad",  plane="x1x2", T=1.0)
plot_trajectory_pair(A, A_t, x0_bad,  "Trajectories from x0_bad",  plane="x2x3", T=1.0)


In [ ]:

# One-parameter family x0(θ) = cosθ v2 + sinθ v3
thetas = np.linspace(0, np.pi/2, 61)
mu_vals, dF_vals = [], []
for theta in thetas:
    x0 = np.cos(theta)*v2 + np.sin(theta)*v3
    mu_vals.append(mu_min(A, B, x0))
    dF_vals.append(d_frob_structured(A, B, x0)[0])
mu_vals = np.array(mu_vals); dF_vals = np.array(dF_vals)

# Plot μ_min(θ)
fig = plt.figure()
ax = plt.gca()
ax.plot(thetas, mu_vals)
ax.set_xlabel("θ (rad)"); ax.set_ylabel("μ_min")
ax.set_title("μ_min vs θ")
ax.grid(True)
plt.show()

# Plot d_Frob(θ)
fig = plt.figure()
ax = plt.gca()
ax.plot(thetas, dF_vals)
ax.set_xlabel("θ (rad)"); ax.set_ylabel("d_Frob(A,B || x0(θ))")
ax.set_title("Structured Frobenius distance vs θ")
ax.grid(True)
plt.show()



### Takeaway

All four proxies \(\{d_\mathrm{Frob},~\mu_{\min},~\sigma_{\min}(K),~\lambda_{\min}(W_T)\}\) are strictly positive
iff PBH holds for all eigenvalues (here, for \(x_0^{\text{good}}\)), and they collapse toward \(0\) when a left eigenvector is orthogonal to both \(B\) and \(x_0\) (here, for \(x_0^{\text{bad}}\) with the \(\lambda_2\) mode hidden).  
The trajectory overlays confirm the equivalence-class argument: \((A,B)\) and \((\tilde A, B)\) produce **identical trajectories** from \(x_0^{\text{bad}}\) and **diverge immediately** from \(x_0^{\text{good}}\).
